# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [40]:
import pandas as pd
import pymongo

In [41]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [42]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [43]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])
 

df_youtube.rename(columns={'video_id':'_id'}, inplace=True)
 
df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')
 
l_d =df_youtube.to_dict(orient='records')
 
import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
 
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [44]:
from pymongo import MongoClient

client = MongoClient("mongo:27017")

db_yt = client.yt

collection_yt = db_yt['videos']

collection_yt.delete_many({})

collection_yt.insert_many(l_d)

## Question 1  

In [45]:
cur = collection_yt.find({"channel_title":"Apple"})
next(cur)

{'_id': 'K4wEI5zhHB0',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'category_id': 28,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
 'views': 19707391,
 'likes': 381919,
 'dislikes': 69465,
 'comment_total': 0,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Science & Technology'}

## Question 2

In [46]:
cur = collection_yt.aggregate([{"$group" : {"_id" : "$category_id", "numbers" : {"$sum" : 1}}}])
list(cur)

[{'_id': 15, 'numbers': 38},
 {'_id': 22, 'numbers': 256},
 {'_id': 26, 'numbers': 262},
 {'_id': 17, 'numbers': 149},
 {'_id': 23, 'numbers': 214},
 {'_id': 2, 'numbers': 37},
 {'_id': 20, 'numbers': 28},
 {'_id': 43, 'numbers': 1},
 {'_id': 24, 'numbers': 477},
 {'_id': 19, 'numbers': 18},
 {'_id': 27, 'numbers': 94},
 {'_id': 1, 'numbers': 101},
 {'_id': 29, 'numbers': 5},
 {'_id': 10, 'numbers': 335},
 {'_id': 25, 'numbers': 198},
 {'_id': 28, 'numbers': 151}]

## Question 3

In [47]:
for video in l_d:
  video['tags']= video['tags'].split('|')
collection_yt.update_one({"_id":{"$eq":video['_id']}},{ "$set":{"tags":video['tags']}})
req = collection_yt.find()
list(req)

[{'_id': '5ywKal6-anc',
  'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ',
  'channel_title': 'TMZ',
  'category_id': 24,
  'tags': 'TMZ2016FS11221|TMZ|Hollywood|Celebrity|Entertainment|Famous|Hollywood News|Fame|Entertainment News',
  'views': 703750,
  'likes': 2921,
  'dislikes': 2196,
  'comment_total': 1042,
  'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Entertainment'},
 {'_id': 'GGm0FQ6i74U',
  'title': 'What Hillary Clinton really thinks',
  'channel_title': 'Vox',
  'category_id': 25,
  'tags': 'vox.com|vox|explain|ezra klein|hillary clinton|what happened|donald trump|election 2016|election 2018|bernie sanders|vox conversations|hillary clinton book|hillary clinton interview|hillary clinton book interview|what happened book|clinton|hillary|maga|make america great again|universal basic income|universal health care|single-payer|health care|hillary rod

## Question 4

In [26]:
cur = collection_yt.find({}).sort([("views", -1)])
next(cur)

{'_id': 'MBdVXkSdhwU',
 'title': "BTS (방탄소년단) 'DNA' Official MV",
 'channel_title': 'ibighit',
 'category_id': 10,
 'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
 'views': 41500672,
 'likes': 2010366,
 'dislikes': 78076,
 'comment_total': 736179,
 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Music'}

## Question 5

In [16]:
req = collection_yt.aggregate([{"$group" : {"_id" : "$category_id", "views" : {"$avg" : "$views"}}}])
list(req)

[{'_id': 19, 'views': 426492.22222222225},
 {'_id': 24, 'views': 1059427.7735849055},
 {'_id': 2, 'views': 622727.8648648649},
 {'_id': 20, 'views': 777213.0357142857},
 {'_id': 17, 'views': 618074.711409396},
 {'_id': 23, 'views': 1317200.7289719626},
 {'_id': 22, 'views': 892992.28515625},
 {'_id': 15, 'views': 633906.4210526316},
 {'_id': 26, 'views': 556002.6679389313},
 {'_id': 43, 'views': 8726.0},
 {'_id': 25, 'views': 533575.1262626263},
 {'_id': 28, 'views': 861563.6158940397},
 {'_id': 10, 'views': 1442240.943283582},
 {'_id': 29, 'views': 681178.2},
 {'_id': 1, 'views': 966547.2871287129},
 {'_id': 27, 'views': 550585.2659574468}]

## Question 6 

In [24]:
req = collection_yt.aggregate([{"$group" : {"_id" : "$channel_title", "avg likes" : {"$avg" : "$likes"}}},  {"$sort" : { "avg likes" : -1 }}])
list(req)

[{'_id': 'ibighit', 'avg likes': 1533457.0},
 {'_id': 'ZaynVEVO', 'avg likes': 1431683.0},
 {'_id': 'BETNetworks', 'avg likes': 982842.0},
 {'_id': 'melanie martinez', 'avg likes': 921577.0},
 {'_id': 'jypentertainment', 'avg likes': 793387.0},
 {'_id': 'Logan Paul Vlogs', 'avg likes': 724478.5},
 {'_id': 'Wengie', 'avg likes': 656526.0},
 {'_id': 'Alesso', 'avg likes': 618436.0},
 {'_id': 'shakiraVEVO', 'avg likes': 578266.0},
 {'_id': 'TaylorSwiftVEVO', 'avg likes': 508015.0},
 {'_id': 'Ed Sheeran', 'avg likes': 461378.0},
 {'_id': 'PewDiePie', 'avg likes': 438273.0},
 {'_id': 'jbalvinVEVO', 'avg likes': 430513.0},
 {'_id': 'Dude Perfect', 'avg likes': 406305.5},
 {'_id': 'Linkin Park', 'avg likes': 398487.0},
 {'_id': 'Liza Koshy', 'avg likes': 397011.2},
 {'_id': 'Conor Maynard', 'avg likes': 389273.0},
 {'_id': 'Bhad Bhabie', 'avg likes': 379224.0},
 {'_id': 'Gabbie Hanna', 'avg likes': 359882.0},
 {'_id': 'Primitive Technology', 'avg likes': 348653.0},
 {'_id': 'ImagineDragonsVEV